In [0]:
                              a)	SparkSession and SparkContext
                              b)	Creating DataFrames
                              c)	DataFrame Operations
                              d)	Running SQL Queries
                              e)	Global Temporary View
                              f)	Aggregations


In [0]:
#Creating SparkSession and SparkContext
#The entry point into all functionality in Spark is the SparkSession class. To create a basic SparkSession, just use SparkSession.builder:

from pyspark.sql import SparkSession

In [0]:
spark = SparkSession \
    	.builder \
    	.appName("Python Spark SQL basic example") \
   	.config("spark.some.config.option", "some-value") \
   	 .getOrCreate()

In [0]:
df = spark.read.json("/FileStore/tables/people-1.json") 
df.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [0]:
df.printSchema()

root
 |-- age: long (nullable = true)
 |-- name: string (nullable = true)



In [0]:
df.select("name").show()

+-------+
|   name|
+-------+
|Michael|
|   Andy|
| Justin|
+-------+



In [0]:
df.select(df['name'], df['age'] + 1).show()

+-------+---------+
|   name|(age + 1)|
+-------+---------+
|Michael|     null|
|   Andy|       31|
| Justin|       20|
+-------+---------+



In [0]:
df.filter(df['age'] > 21).show()

+---+----+
|age|name|
+---+----+
| 30|Andy|
+---+----+



In [0]:
df.groupBy("age").count().show()

+----+-----+
| age|count|
+----+-----+
|  19|    1|
|null|    1|
|  30|    1|
+----+-----+



In [0]:
#The sql function on a SparkSession enables applications to run SQL queries programmatically and returns the result as a DataFrame.
df.createOrReplaceTempView("people")

sqlDF = spark.sql("SELECT * FROM people")
sqlDF.show()


+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [0]:
#Temporary views in Spark SQL are session-scoped and will disappear if the session that creates it terminates. 
#Register the DataFrame as a global temporary view
df.createGlobalTempView("people")


In [0]:
#Global temporary view is tied to a system preserved database `global_temp`
spark.sql("SELECT * FROM global_temp.people").show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [0]:
#Global temporary view is cross-session
spark.newSession().sql("SELECT * FROM global_temp.people").show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [0]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import approx_count_distinct,collect_list
from pyspark.sql.functions import collect_set,sum,avg,max,countDistinct,count
from pyspark.sql.functions import first, last, kurtosis, min, mean, skewness 
from pyspark.sql.functions import stddev, stddev_samp, stddev_pop, sumDistinct
from pyspark.sql.functions import variance,var_samp,  var_pop

In [0]:
#Creating SparkSession

spark = SparkSession.builder.appName('SparkByExamples.com').getOrCreate()

In [0]:
#Sample Dataset
  
simpleData = [("James", "Sales", 3000),
    ("Michael", "Sales", 4600),
    ("Robert", "Sales", 4100),
    ("Maria", "Finance", 3000),
    ("James", "Sales", 3000),
    ("Scott", "Finance", 3300),
    ("Jen", "Finance", 3900),
    ("Jeff", "Marketing", 3000),
    ("Kumar", "Marketing", 2000),
    ("Saif", "Sales", 4100)]
schema = ["employee_name", "department", "salary"]


In [0]:
df = spark.createDataFrame(data=simpleData, schema = schema)
df.printSchema()
df.show(truncate=False)


root
 |-- employee_name: string (nullable = true)
 |-- department: string (nullable = true)
 |-- salary: long (nullable = true)

+-------------+----------+------+
|employee_name|department|salary|
+-------------+----------+------+
|James        |Sales     |3000  |
|Michael      |Sales     |4600  |
|Robert       |Sales     |4100  |
|Maria        |Finance   |3000  |
|James        |Sales     |3000  |
|Scott        |Finance   |3300  |
|Jen          |Finance   |3900  |
|Jeff         |Marketing |3000  |
|Kumar        |Marketing |2000  |
|Saif         |Sales     |4100  |
+-------------+----------+------+



In [0]:
#Calculate approx_count_distinct
# In PySpark approx_count_distinct() function returns the count of distinct items in a group.
print("approx_count_distinct: " + \
     str(df.select(approx_count_distinct("salary")).collect()[0][0]))


approx_count_distinct: 6


In [0]:
# Calculate average
# avg() function returns the average of values in the input column.
print("avg: " + str(df.select(avg("salary")).collect()[0][0]))


avg: 3400.0


In [0]:
#Calculate list
#collect_list() function returns all values from an input column with duplicates.
df.select(collect_list("salary")).show(truncate=False)


+------------------------------------------------------------+
|collect_list(salary)                                        |
+------------------------------------------------------------+
|[3000, 4600, 4100, 3000, 3000, 3300, 3900, 3000, 2000, 4100]|
+------------------------------------------------------------+



In [0]:
#Calculate countDistinct
#countDistinct() function returns the number of distinct elements in a columns
df2 = df.select(countDistinct("department", "salary"))
df2.show(truncate=False)
print("Distinct Count of Department & Salary: "+str(df2.collect()[0][0]))


+----------------------------------+
|count(DISTINCT department, salary)|
+----------------------------------+
|8                                 |
+----------------------------------+

Distinct Count of Department & Salary: 8


In [0]:
#Show columns
# count() function returns number of elements in a column.
print("count: "+str(df.select(count("salary")).collect()[0]))


count: Row(count(salary)=10)


In [0]:
#Show First Element in Column
# first() function returns the first element in a column when ignoreNulls is set to true, it returns the first non-null element.
df.select(first("salary")).show(truncate=False)


+-------------+
|first(salary)|
+-------------+
|3000         |
+-------------+



In [0]:
#Show Last Element in Column
# last() function returns the last element in a column. when ignoreNulls is set to true, it returns the last non-null element.
df.select(last("salary")).show(truncate=False)


+------------+
|last(salary)|
+------------+
|4100        |
+------------+



In [0]:
#Show maximum value in a column.
# max() function returns the maximum value in a column.
df.select(max("salary")).show(truncate=False)


+-----------+
|max(salary)|
+-----------+
|4600       |
+-----------+



In [0]:
#Show minimum value in a column.
# min() function
df.select(min("salary")).show(truncate=False)


+-----------+
|min(salary)|
+-----------+
|2000       |
+-----------+



In [0]:
#Show mean value in a column.
# mean() function returns the average of the values in a column. Alias for Avg
df.select(mean("salary")).show(truncate=False)


+-----------+
|avg(salary)|
+-----------+
|3400.0     |
+-----------+



In [0]:
#Calculate Standard Deviation
# stddev() alias for stddev_samp.
# stddev_samp() function returns the sample standard deviation of values in a column.
# stddev_pop() function returns the population standard deviation of the values in a column.

df.select(stddev("salary"), stddev_samp("salary"), stddev_pop("salary")).show(truncate=False)


+-------------------+-------------------+------------------+
|stddev_samp(salary)|stddev_samp(salary)|stddev_pop(salary)|
+-------------------+-------------------+------------------+
|765.9416862050705  |765.9416862050705  |726.636084983398  |
+-------------------+-------------------+------------------+



In [0]:
#Calculate sum of all values in a column.
# sum() function Returns the sum of all values in a column
df.select(sum("salary")).show(truncate=False)


+-----------+
|sum(salary)|
+-----------+
|34000      |
+-----------+



In [0]:
#Calculate variance of the values in a column.
# variance() alias for var_samp
# var_samp() function returns the unbiased variance of the values in a column.
# var_pop() function returns the population variance of the values in a column

df.select(variance("salary"),var_samp("salary"),var_pop("salary"))  .show(truncate=False)


+-----------------+-----------------+---------------+
|var_samp(salary) |var_samp(salary) |var_pop(salary)|
+-----------------+-----------------+---------------+
|586666.6666666666|586666.6666666666|528000.0       |
+-----------------+-----------------+---------------+

